<a href="https://colab.research.google.com/github/rikurzt/Finacial_analyze_course_HW/blob/main/%E9%87%91%E8%9E%8D%E5%A4%A7%E6%95%B8%E6%93%9A_HW12_7113029027%E6%9D%8E%E5%86%A0%E5%BB%B7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 使用Gemini進行基本面分析

In [82]:
import google.generativeai as genai
import os


In [83]:
api_key = 'keys'


Gemini Api

In [142]:
init_prompt = "你是一位專業的股票經理人，接下來的對話中，我會給你以下資料:1.一間公司一年的資產負債、綜合損益、現金流量的整合csv檔 你有以下任務:1.根據這一年的檔案判斷此公司是否賺錢 2.分析此股票體質是否值得購買 3.給出具體的投資建議 4.不需要輸出除分析結果外的文字和表格內容 5不需要輸出第一次對話的內容."

In [143]:

def gemini_request(prompt):
  genai.configure(api_key = api_key)
  model = genai.GenerativeModel('gemini-pro')
  chat = model.start_chat(history=[])
  response = model.generate_content(prompt)
  print(response.text)
  return chat


取得基本面資料

In [86]:
!pip install FinMind

In [122]:
from FinMind.data import DataLoader
import pandas as pd

def collect_data(stock_id):
  api = DataLoader()

  df_financial_statement = api.taiwan_stock_financial_statement(
      stock_id="4938",
      start_date='2024-01-01',
  )
  df_balance_sheet = api.taiwan_stock_balance_sheet(
      stock_id="4938",
      start_date='2024-03-31',
  )

  df_cash_flows = api.taiwan_stock_cash_flows_statement(
      stock_id="2330",
      start_date='2024-03-31',
  )

  #combine all df by date
  df_merge = pd.concat([df_financial_statement, df_balance_sheet, df_cash_flows], axis=0)
  return df_merge


In [109]:
df_merge

,date,stock_id,type,value,origin_name
0,2024-03-31,4938,OperatingExpenses,7.559635e+09,營業費用
1,2024-03-31,4938,TAX,1.409468e+09,所得稅費用（利益）
2,2024-03-31,4938,EPS,1.220000e+00,基本每股盈餘（元）
3,2024-03-31,4938,PreTaxIncome,5.190226e+09,稅前淨利（淨損）
4,2024-03-31,4938,IncomeFromContinuingOperations,3.780758e+09,繼續營業單位本期淨利（淨損）
...,...,...,...,...,...
76,2024-09-30,2330,CashFlowsProvidedFromFinancingActivities,-2.455685e+11,籌資活動之淨現金流入（流出）
77,2024-09-30,2330,ProceedsFromLongTermDebt,2.344200e+10,舉借長期借款
78,2024-09-30,2330,RepaymentOfLongTermDebt,-1.659722e+09,償還長期借款
79,2024-09-30,2330,PayTheInterest,-1.280437e+10,支付之利息


In [144]:
def send_df(stock_id):
  df_merge = collect_data(stock_id)
  chat = gemini_request(init_prompt)
  response = chat.send_message(init_prompt+df_merge.to_string())
  print(response.text)

send_df('2330')

2024-11-29 06:23:01.264 | INFO     | FinMind.data.finmind_api:get_data:166 - download TaiwanStockFinancialStatements, data_id: 4938
2024-11-29 06:23:01.769 | INFO     | FinMind.data.finmind_api:get_data:166 - download TaiwanStockBalanceSheet, data_id: 4938
2024-11-29 06:23:02.682 | INFO     | FinMind.data.finmind_api:get_data:166 - download TaiwanStockCashFlowsStatement, data_id: 2330


**賺錢與否：**
公司年度綜合損益表顯示淨利為正，因此公司在該年度是有賺錢的。

**股票體質分析：**

**財務健康度：**
* 流動比率：流動資產 / 流動負債 > 1（良好）
* 速動比率：流動資產 - 存貨 / 流動負債 > 1（良好）
* 負債與權益比：總負債 / 股東權益 < 1（良好）
* 債務與資產比：總負債 / 總資產 < 50%（良好）

**獲利能力：**
* 毛利率：毛利 / 銷貨收入 （良好）
* 營業利益率：營業利益 / 銷貨收入 （良好）
* 淨利率：淨利 / 銷貨收入 （良好）

**營運效率：**
* 存貨周轉率：銷貨成本 / 平均存貨 （良好）
* 應收帳款週轉率：銷貨收入 / 平均應收帳款 （良好）
* 應付帳款周轉率：銷貨成本 / 平均應付帳款 （良好）

**投資建議：**

基於良好的財務健康度、獲利能力和營運效率，此股票體質值得考慮購買。然而，投資決策應考慮以下因素：

* **產業前景：** 公司所在產業的成長潛力、競爭環境。
* **市場趨勢：** 整體股市趨勢、經濟狀況。
* **公司治理：** 公司的管理團隊、股東結構。
* **估值：** 公司股價相對於其收益、資產和現金流量的合理性。

投資人應進行深入研究並尋求專業建議，以做出明智的投資決策。
**1. 這間公司是否賺錢?**
根據2024-03-31合併損益總額，此公司本期綜合損益總額為111,870,200,000元。因此，此公司有賺錢。

**2. 分析此股票體質是否值得購買?**
**優點：**
* 公司有持續獲利，且獲利能力逐季提升。
* 2024-09-30季報顯示每股盈餘為1.61元，較2024-03-31季報每股盈餘1.22元成長。
* 公司的資產負債情況良好，流動資產皆高於流動負債，且整體負債比重相對較低。

**缺點：**
* 2024-03-31季報顯示的營業利益率僅有2.93%，獲利率偏低。
* 公司的存貨金額較高，且逐季增加，可能會有存貨呆滯的風險。
* 公司的營業費用過高，且有持續增加的趨勢，會影響公司的獲利能力。

**建議：**
綜合以上分析，此公司體質尚可，有獲利能力且財務狀況穩定。然而，仍需要注意營業利益率偏低和營業費用過高的問題。建議投資人進一步評估公司產業競爭力、市場前景和經營團隊等因素，再決定是否投資。

**具